In [ ]:
%matplotlib inline
# Importing necessary packages:
import re
from glob import glob

import scipy.integrate as integrate

import numpy as np
import datetime
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time
import tracemalloc
import cProfile

from PolyLine import *


import scipy.integrate as integrate
import scipy.special as special
from sympy import elliptic_pi

np.set_printoptions(precision=17)

In [ ]:
static_proeprties = pd.read_csv("./runs_stat_20200617_all_up_to_this_date.txt",comment='#')
static_proeprties[(static_proeprties['nmon']==2000) & (static_proeprties['dmon']==1) &  (static_proeprties['dcyl']==21) & (static_proeprties['vfrc_c']==0.05)& (static_proeprties['dcrowd']==1)].chrm_r.mean()

In [ ]:
hist_files = glob("../N2000D20ac2-analyze-not_full_phic-analyze/N*.txt")
hist_files = PolyLine.file_reader(hist_files,extensions=['_rEdges.txt','_rHists.txt'])

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
nens = 0
hists = 0
phi_c = 0.0
for hist_pair in hist_files:
    cell_attrs = PolyLine.cellAttributes(hist_pair[0],geometry='cylinder')
    #print(round(cell_attrs.vfrc_crowd,3))
    if cell_attrs.vfrc_crowd == phi_c :
        nens = nens + 1
        hists = hists + np.loadtxt(hist_pair[0])
        edges = np.loadtxt(hist_pair[1])

hists = hists / nens
PolyLine.histo_plotter(ax, hists, lambda r: 2 * cell_attrs.dcyl / 2.0 * r, edges, label = r"$\phi_c={}$".format(phi_c), markersize=0.1)
ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")
print(nens)

In [ ]:
def local_number_density(histograms, bin_edges, integrand):
    bin_size = np.around(bin_edges[1] - bin_edges[0],decimals=2) # assuming bins with an equal size
    bin_vols = [integrate.quad(integrand, bin_min ,bin_min + bin_size)[0] for bin_min in bin_edges[:-1]]
    histograms = histograms / np.sum(histograms) # normalization = the sum of histograms = natoms * nframes

    rho = np.divide(histograms, bin_vols)
    # the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
    # this arises from the way we descritize the local number desnity.
    rho = rho / np.sum(rho)
    return rho

In [ ]:
def bin_edge_index(ratom, positions, r_concentrics):
    """
    bin_edge_index finds the number of r_concentrics spheres/circles all centered at the origin 
    (each two consecutive r_concentrics create a bin) that an atom of radius r_atom located at bin_center
    along axis x intersects with. The problem is a 1D problem becuase of the way we choose the positions 
    of the atom and the concenteric objects.
    
    The positions and r_concentrics are given in xy-plane (polar coordinates).
    
    The positions and r_concentrics have some relations:
    1. len(r_concentrics) = len(positions) + 1
    2. There is only one position between two consecutive r_concentrics.
    3. If r_concentrics are the edges of histograms bins, the lower_lim_idx is the index of the inner edge
    of the innermost bin and upper_lim_idx is the index of the inner edge of the outermost bin.
    4. This algorithm depends on the binsize=0.1=r_atom/10=1/10 which is defined in the PipeLine.trj_analyze
    so it works only for monomers with diameters that are multiple of amon=1.
    
    Inputs:
    ratom (float): the radius of the spherical partilce.
    positions (numpy array): positions of the center of the particle along axis x.
    r_concentrics (numpy array): the radii of the concentric spheres/circles/cylinders all centered at the origin.
    
    Return: 
    range_of_bins (dict): the dictionary that has the index of positions as keys and the below-explained
    tuples as its values. Each tuple contains the indexes of the lowest and highest r_concentrics 
    between which the position vector (that is give by the key) is located.
    
    Requirements:
    Numpy package
    
    """
    r_concentrics_idx_range = {}
    for pos_idx in range(len(positions)):
        
        innermost_pos = positions[pos_idx] - ratom # The minimum distance of the r_atoms perimeter from the origin
        if innermost_pos <= 0:
            innermost_idx = 0
        else:
            for r_con_idx in range(len(r_concentrics)-1):
                if (innermost_pos >= r_concentrics[r_con_idx]) and (innermost_pos < r_concentrics[r_con_idx+1]):
                    innermost_idx = r_con_idx   
        
        outermost_pos = positions[pos_idx] + ratom # The maximum distance of the r_atoms perimeter from the origin
        for r_con_idx in range(len(r_concentrics)-1):
            if (outermost_pos >= r_concentrics[r_con_idx]) and (outermost_pos < r_concentrics[r_con_idx+1]):
                outermost_idx = r_con_idx
                if outermost_idx > (len(r_concentrics)-1):
                    outermost_idx = len(r_concentrics)-1
        r_concentrics_idx_range[pos_idx] = (innermost_idx, outermost_idx)
        # volume of intersection of a spherical bead of diameter dmon located at x = bin_center and,
        # a sphere of radius bin_edge located at the origin:
    return r_concentrics_idx_range

In [ ]:
def sphere_sphere_intersction(r, R, d):
    """
    sphere_sphere_intersction computes the volume of intersection of two spheres. The sphere with redius R
    is at the origin (0,0,0) while the other one with radius r is located along axis x at x=d (d,0,0).
    
    This function can be used to find the local volume fraction of a spherical beads in a space with spherical
    symmetry.
    
    
    Reference: https://mathworld.wolfram.com/Sphere-SphereIntersection.html
    
    Inputs:
    r: the radius of the sphere locared along axis x.
    R: the radius of the sphere located at the origin.
    d: the distance of the the off-origin sphere from the origin along axis x.
    
    Returns:
    V: volume of intersection
    
    Requirements:
    numpy package for constant Pi.
    """
    
    # By define Rmax and Rmin, we handlthe situtations in which d = 0:
    
    Rmax = max(R,r)
    Rmin = min(R,r)
    
    if r ==0 or R == 0:
        V = 0 
    else : 
        if d == 0: # the small sphere resides completely in the large one.
            V = 4*np.pi*Rmin**3 / 3 

        elif d >= Rmin+Rmax: # the spheres are either tengential to eachother or not interesting.
            V = 0
        else:
            if d <= Rmax-Rmin: # the small sphere resides completely in the large one.
                V = 4*np.pi*Rmin**3 / 3

            else :
                V = np.pi * (Rmax + Rmin - d)**2 * (d**2 + 2*d*Rmin - 3*Rmin**2 + 2*d*Rmax + 6*Rmin*Rmax - 3*Rmax**2) / (12*d)
    
    return V

In [ ]:
def sphere_cylinder_intersection(R , r, b):
    """
    computes the volume of intersection of  a sphere and a cylinder. This function is written 
    based on the following article:
    https://www.sciencedirect.com/science/article/pii/0010465590901843
    
    Parameters:
    R: the radius of a sphere
    r: the radius of a cylinder
    b: the smallest distance between the centers of the sphere and the axis of the cylinder
    
    Caution:
    r here is the radius of cylinder but it is the radius of sphere in the article.
    
    Returns:
    V_i: the volume of intersection
    """
    xmax = b + R 
    xmin = b - R
    if r > xmax: # For the purpose of the local volume fraction calculation, this case is meaningful.
        #raise Exception("This funtion only works for the systems where the cylinder's radius is smaller than the sphere radius.")
        V_in = 0
    if 0 <= xmin - r: # There  is no intersection if R =< d-r. This works only if d resides on x+ axis.
        #raise Exception('R <= (b-r): There is no intersection.')
        V_in = 0
    else:
        def intersec_cons(R, r, b):
            """
            computes the quantities needed for calculating the volume of intersection of a sphere and a cylinder. This
            function is written based on the following article:
            https://www.sciencedirect.com/science/article/pii/0010465590901843

            Parameters:
            R: the radius of a sphere
            r: the radius of a cylinder
            b: the smallest distance between the centers of the sphere and the axis of the cylinder

            Caution:
            This implementation of the intersection volume is used to computing the volume fraction of several
            spherical monomers in a cylinderical geometry (the bins are cylindrical shells). As a result, r is
            always smaller than R+b: r < R + b

            Returns:
            A: the maximum of R^2 and (r+b)^2
            B: the minimum of R^2 and (r+b)^2
            C: (b-r)^2
            m: the parameter of the elliptic integral (In the main reference, it is shown by k^2 but in Python's 
            scipy package and Wolfram Mathematica the m is used. As a result m=k^2). k is called the elliptic moduleus
            while m is called

            n: the second parameter of the elliptic integral of the third kind. In the main article, it is shown by
            -\alpha^2. Pay attention to the location of the -\alpha^2 in the equation of the elliptic integral of the
            third kind in the article.
            s: (b+r)(b-r)

            """
            A = max(R ** 2, (b + r) ** 2)
            #print ("A:", A)
            B = min(R ** 2, (b + r) ** 2)
            #print ("B:", B)
            C = (b - r) ** 2
            #print ("C:", C)

            if A == C and A != B: # According to my calculation, if A != B and A == C, then k2=+inf.
                k2 = np.inf
                print ("k2:", k2)
            elif A == C and A == B: 
                k2 = np.nan
                #print("k2 cannot be measured.")
            else:
                k2 = (B - C) / (A - C)
                #print ("k2:", k2)

            if C == 0.:
                neg_alpha2 = np.inf
                #print ("neg_alpha2:", neg_alpha2)
            else:
                neg_alpha2 = (B - C) / C # -\alpa^2 in the article
                #print ("neg_alpha2:", neg_alpha2)
            s = (b + r) * (b - r)
            #print ("s:", s)
            return (A, B, C, k2, neg_alpha2, s)

        A, B, C, k2, neg_alpha2, s = intersec_cons(R, r, b)
        V_1 = 4 / 3 * np.pi * (R ** 3) * np.heaviside(r - b, 1.)

        if R > (b + r):
            if C==0.:
                V_2_pi = A ** 2 * s * elliptic_pi(-neg_alpha2, k2).evalf() / C 
                #print('V_2_pi:',V_2_pi)
            else:
                V_2_pi = A ** 2 * s * elliptic_pi(-neg_alpha2, k2).evalf() / C 
            # evalf method converts the result to floating point numerics
            V_2_k = (A * s - (A - B) * (A - C) / 3) * special.ellipk(k2)
            V_2_e = (A - C) * (s + (4 * A - 2 * B - 2 * C) / 3) * special.ellipe(k2)
            V_2 = 4 / (3 * np.sqrt(A - C)) * (V_2_pi - V_2_k - V_2_e)
            V_in = V_1 + V_2
        elif R < (b + r):
            if C==0.:
                V_2_pi = B ** 2 * s * elliptic_pi(-neg_alpha2, k2).evalf() / C
                #print('V_2_pi:',V_2_pi)
            else:
                V_2_pi = B ** 2 * s * elliptic_pi(-neg_alpha2, k2).evalf() / C
            V_2_k = ((A - 2 * B) * s + (A - B) * (3 * B - C - 2 * A) / 3) * special.ellipk(k2)
            V_2_e = (A - C) * (- s + (2 * A + 2 * C - 4 * B) / 3) * special.ellipe(k2)
            V_2 = 4 / (3 * np.sqrt(A - C)) * (V_2_pi + V_2_k + V_2_e)
            V_in = V_1 + V_2
        else :
            # np.arctan or np.arctan2: ?
            # There is warning here if C=0 due to the division by zero in arctan's argument.
            V_2_arc = 4 * R ** 3 / 3 * np.arctan(2 * np.sqrt(b * r) / (b - r)) 
            V_2_sqrt = 4 / 3 * np.sqrt(A - C) * (s + 2 / 3 * (A - C))
            V_2 = V_2_arc - V_2_sqrt
            V_in = V_1 + V_2
            
    return V_in

In [ ]:
def vol_shares(ratom, positions, r_concentrics, range_of_bins, intersection_calculator):
    """
    vol_shares computes the portion of the volume of a spherical bead in different concenteric bins. 
    The func computes the volume of intersection between a spherical bead and an object (sphere or 
    cylinder) of radius r_concentric located at origin.
    The volume share of the innermost bin is equal to the volume of the intersection given by the func.
    The volume share of the other bins is equal to the volume of the intersection of the bin with the 
    inner edge of radius r_concentric substracted by the volume of the intersection of the previous bin 
    with the inner edge given by its radius r_concentric.
    
    
    Inputs:
    ratom (float): the radius of the spherical partilce.
    positions (numpy array): positions of the center of the particle along axis x.
    r_concentrics (numpy array): the radii of the concentric spheres/circles/cylinders all centered at the origin.
    range_of_bins (dict): the dictionary that has the index of positions as keys and the below-explained
    tuples as its values. Each tuple contains the indexes of the lowest and highest r_concentrics 
    between which the position vector (that is give by the key) is located.
    intersection_calculator: the function computes the volume of intersection between the spherical bead and concentric
    shells given by radii r_concentrics in xy-plane (polar coordinates).
    
    Return: 
    volume_shares (dict): The array of the volume shares of bins with inner edges of radius 
    r_concentrics.
    
    Requirements:
    sphere_sphere_intersction function from the PolyPipe.
    Numpy package.
    """
    volume_shares = {}
    for pos_idx, edge_limits in range_of_bins.items():
        volume_shares[pos_idx] = {}
        intersect_vol_previous = 0 # The volume share of the lowest bin all comes from itself.
        for idx in range(edge_limits[0]+1,edge_limits[1]+1,1):
            intersect_vol = intersection_calculator(ratom, r_concentrics[idx], positions[pos_idx])
            volume_shares[pos_idx][idx]= intersect_vol - intersect_vol_previous # The volume share of the previous bin should be subsracted
            intersect_vol_previous = intersect_vol
            
    return volume_shares

In [ ]:
def local_volume_fraction(bin_centers, rho, volume_shares):
    phi = np.zeros(len(rho))
    for bin_center_idx in range(len(bin_centers)):
        for vol_share_idx, vol_share_value in volume_shares[bin_center_idx].items():
            phi[bin_center_idx] = phi[bin_center_idx] + (rho[vol_share_idx] * (vol_share_value))
    # the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
    # this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
    phi = phi / np.sum(phi)
    return phi

In [ ]:
fpath = hist_files[24]

histo_collections = np.loadtxt(fpath[0],dtype=np.int32)
bin_edges = np.around(np.loadtxt(fpath[1]),decimals=2)
cell_attrs = PolyLine.cellAttributes(fpath[0],geometry='cylinder',printname=True)
filename = cell_attrs.filename
cylindrical_shell_integrand = lambda r: 2 * np.pi * cell_attrs.dcyl * r

# the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
# this arises from the way we descritize the local number desnity.
rho = local_number_density(histo_collections, bin_edges, cylindrical_shell_integrand)

rmon = cell_attrs.dmon / 2.0
bin_centers = np.around((bin_edges[:-1] + bin_edges[1:]) / 2.0,decimals=2)

range_of_bins = bin_edge_index(rmon, bin_centers, bin_edges)
volume_shares = vol_shares(rmon, bin_centers, bin_edges, range_of_bins, sphere_cylinder_intersection)

# the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
# this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
phi = local_volume_fraction(bin_centers, rho, volume_shares)

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))

fontsize = 24
color = 'tab:blue'
line1, = ax.plot(bin_centers, phi, label = r"$\phi_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(bin_centers, rho, label = r"$\rho_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)